In [1]:
import os
import csv
import pickle
import random
import math
import dicom
import numpy as np
import scipy
from tqdm import tqdm
from tinydb import TinyDB, Query
from natsort import natsorted
from skimage import transform
from sklearn.externals import joblib

import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image

#### load and shuffle data

In [2]:
(pts_train, pts_train_val, 
 data_train, data_diffs_train, label_sys_train, label_dia_train, 
 data_train_val, data_diffs_train_val, label_sys_train_val, label_dia_train_val, 
 data_val, data_diffs_val, data_val_pt_index) = joblib.load('../data_proc/0-data_processed.pkl')

In [3]:
shuffle_index = list(range(data_train.shape[0]))
random.shuffle(shuffle_index)
data_train = data_train[shuffle_index]
label_sys_train = label_sys_train[shuffle_index]
label_dia_train = label_dia_train[shuffle_index]

shuffle_index = list(range(data_train_val.shape[0]))
random.shuffle(shuffle_index)
data_train_val = data_train_val[shuffle_index]
label_sys_train_val = label_sys_train_val[shuffle_index]
label_dia_train_val = label_dia_train_val[shuffle_index]

#### training

In [5]:
from keras.models import Sequential, Graph
from keras.layers.core import Activation, Dense, Dropout, Flatten, Merge, Reshape, Lambda
from keras.layers.core import TimeDistributedDense, TimeDistributedMerge
from keras.layers.recurrent import LSTM, GRU
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D, UpSampling2D, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU, PReLU, ParametricSoftplus, ELU
from keras.layers.normalization import BatchNormalization
from keras.layers.noise import GaussianDropout, GaussianNoise
from keras.utils import np_utils, generic_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using Theano backend.


Using gpu device 3: Tesla K80 (CNMeM is disabled)


Continuous Ranked Probability Score (used as loss function)

In [6]:
from keras import backend as K
from theano import tensor as T

# currently for theano, would need modifications for tensorflow

def CRPS(y_true, y_pred):
    return K.mean(K.square(T.cumsum(y_pred, axis=-1) - y_true), axis=-1)

In [17]:
model_sys = Sequential()
model_sys.add(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th', input_shape=(30, 196, 196)))
model_sys.add(BatchNormalization())
model_sys.add(ELU())
model_sys.add(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_sys.add(BatchNormalization())
model_sys.add(ELU())
model_sys.add(MaxPooling2D(pool_size=(2,2), strides=None, border_mode='valid', dim_ordering='th'))
model_sys.add(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_sys.add(BatchNormalization())
model_sys.add(ELU())
model_sys.add(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_sys.add(BatchNormalization())
model_sys.add(ELU())
model_sys.add(MaxPooling2D(pool_size=(2, 2), strides=None, border_mode='valid', dim_ordering='th'))
model_sys.add(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_sys.add(BatchNormalization())
model_sys.add(ELU())
model_sys.add(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_sys.add(BatchNormalization())
model_sys.add(ELU())
model_sys.add(MaxPooling2D(pool_size=(2, 2), strides=None, border_mode='valid', dim_ordering='th'))
model_sys.add(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_sys.add(BatchNormalization())
model_sys.add(ELU())
model_sys.add(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_sys.add(BatchNormalization())
model_sys.add(ELU())
model_sys.add(MaxPooling2D(pool_size=(2, 2), strides=None, border_mode='valid', dim_ordering='th'))
model_sys.add(Flatten())
model_sys.add(Dense(1024, activation='relu'))
model_sys.add(Dropout(0.5))
model_sys.add(Dense(600))
model_sys.add(Activation('softmax'))

model_sys.compile(loss=CRPS, optimizer='rmsprop')

In [18]:
batch_size = 32
nb_epoch = 100
    
checkpointer = ModelCheckpoint(filepath='../model_weights/0-1-convnet_basic_systole.hdf5', verbose=1, save_best_only=True)
earlystopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1)

model_sys.fit(data_train, label_sys_train, 
              batch_size=batch_size, nb_epoch=nb_epoch, show_accuracy=False, verbose=2,
              validation_data=(data_train_val, label_sys_train_val), shuffle=True,
              callbacks=[checkpointer, earlystopping])

Train on 5065 samples, validate on 266 samples
Epoch 1/100
93s - loss: 0.1188 - val_loss: 0.0324
Epoch 00000: val_loss improved from inf to 0.03237, saving model to ../model_weights/0-1-convnet_basic_systole.hdf5
Epoch 2/100
93s - loss: 0.0379 - val_loss: 0.0337
Epoch 00001: val_loss did not improve
Epoch 3/100
93s - loss: 0.0370 - val_loss: 0.0328
Epoch 00002: val_loss did not improve
Epoch 4/100
93s - loss: 0.0365 - val_loss: 0.0316
Epoch 00003: val_loss improved from 0.03237 to 0.03164, saving model to ../model_weights/0-1-convnet_basic_systole.hdf5
Epoch 5/100
93s - loss: 0.0360 - val_loss: 0.0310
Epoch 00004: val_loss improved from 0.03164 to 0.03104, saving model to ../model_weights/0-1-convnet_basic_systole.hdf5
Epoch 6/100
93s - loss: 0.0355 - val_loss: 0.0316
Epoch 00005: val_loss did not improve
Epoch 7/100
93s - loss: 0.0352 - val_loss: 0.0302
Epoch 00006: val_loss improved from 0.03104 to 0.03025, saving model to ../model_weights/0-1-convnet_basic_systole.hdf5
Epoch 8/100
9

In [24]:
model_dia = Sequential()
model_dia.add(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th', input_shape=(30, 196, 196)))
model_dia.add(BatchNormalization())
model_dia.add(ELU())
model_dia.add(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_dia.add(BatchNormalization())
model_dia.add(ELU())
model_dia.add(MaxPooling2D(pool_size=(2,2), strides=None, border_mode='valid', dim_ordering='th'))
model_dia.add(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_dia.add(BatchNormalization())
model_dia.add(ELU())
model_dia.add(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_dia.add(BatchNormalization())
model_dia.add(ELU())
model_dia.add(MaxPooling2D(pool_size=(2, 2), strides=None, border_mode='valid', dim_ordering='th'))
model_dia.add(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_dia.add(BatchNormalization())
model_dia.add(ELU())
model_dia.add(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_dia.add(BatchNormalization())
model_dia.add(ELU())
model_dia.add(MaxPooling2D(pool_size=(2, 2), strides=None, border_mode='valid', dim_ordering='th'))
model_dia.add(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_dia.add(BatchNormalization())
model_dia.add(ELU())
model_dia.add(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_dia.add(BatchNormalization())
model_dia.add(ELU())
model_dia.add(MaxPooling2D(pool_size=(2, 2), strides=None, border_mode='valid', dim_ordering='th'))
model_dia.add(Flatten())
model_dia.add(Dense(1024, activation='relu'))
model_dia.add(Dropout(0.5))
model_dia.add(Dense(600))
model_dia.add(Activation('softmax'))

model_dia.compile(loss=CRPS, optimizer='rmsprop')

In [25]:
batch_size = 32
nb_epoch = 100
    
checkpointer = ModelCheckpoint(filepath='../model_weights/0-1-convnet_basic_diastole.hdf5', verbose=1, save_best_only=True)
earlystopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1)

model_dia.fit(data_train, label_dia_train, 
              batch_size=batch_size, nb_epoch=nb_epoch, show_accuracy=False, verbose=2,
              validation_data=(data_train_val, label_dia_train_val), shuffle=True,
              callbacks=[checkpointer, earlystopping])

Train on 5065 samples, validate on 266 samples
Epoch 1/100
93s - loss: 0.1108 - val_loss: 0.0598
Epoch 00000: val_loss improved from inf to 0.05984, saving model to ../model_weights/0-1-convnet_basic_diastole.hdf5
Epoch 2/100
93s - loss: 0.0587 - val_loss: 0.0498
Epoch 00001: val_loss improved from 0.05984 to 0.04984, saving model to ../model_weights/0-1-convnet_basic_diastole.hdf5
Epoch 3/100
93s - loss: 0.0552 - val_loss: 0.0489
Epoch 00002: val_loss improved from 0.04984 to 0.04890, saving model to ../model_weights/0-1-convnet_basic_diastole.hdf5
Epoch 4/100
93s - loss: 0.0548 - val_loss: 0.0473
Epoch 00003: val_loss improved from 0.04890 to 0.04729, saving model to ../model_weights/0-1-convnet_basic_diastole.hdf5
Epoch 5/100
93s - loss: 0.0543 - val_loss: 0.0469
Epoch 00004: val_loss improved from 0.04729 to 0.04686, saving model to ../model_weights/0-1-convnet_basic_diastole.hdf5
Epoch 6/100
93s - loss: 0.0538 - val_loss: 0.0460
Epoch 00005: val_loss improved from 0.04686 to 0.046

#### predict

In [21]:
model_sys.load_weights('../model_weights/0-1-convnet_basic_systole.hdf5')
model_dia.load_weights('../model_weights/0-1-convnet_basic_diastole.hdf5')

preds_sys = model_sys.predict(data_val, verbose=0)
preds_dia = model_dia.predict(data_val, verbose=0)

preds_sys = np.clip(np.cumsum(preds_sys, axis=-1), 0, 1)
preds_dia = np.clip(np.cumsum(preds_dia, axis=-1), 0, 1)

In [22]:
preds_sys_pt = {}
preds_dia_pt = {}
for pt in range(501, 701):
    preds_sys_pt[pt] = np.mean(preds_sys[np.where(data_val_pt_index == pt)[0]], axis=0)
    preds_dia_pt[pt] = np.mean(preds_dia[np.where(data_val_pt_index == pt)[0]], axis=0)

In [23]:
with open('../data/sample_submission_validate.csv', 'r') as fi:
    reader = csv.reader(fi)
    header = next(reader)
    
    with open('../submissions/0-1-convnet_basic.csv', 'w') as fo:
        writer = csv.writer(fo, lineterminator='\n')
        writer.writerow(header)
        for rowin in tqdm(reader):
            _id = rowin[0]
            pt, mode = _id.split('_')
            rowout = [_id]
            if mode.lower() == 'systole':
                rowout.extend(preds_sys_pt[int(pt)].tolist())
            elif mode.lower() == 'diastole':
                rowout.extend(preds_dia_pt[int(pt)].tolist())
            else:
                raise
            writer.writerow(rowout)